### Introduction to Machine Learning - HW3 - Q4
#### Professors: Abolghasemi & Arabi
##### Student: Mohamad Mahdi Samadi
##### Student ID: 810101465

Given $ f(x) = 100\times(x_2 - x_1^2)^2 + (1-x_1)^2 $, answer the following questions.

#### a. calculte gradient vector and hessian matrix for $ f(x) $.

$$ \nabla f(x) = [\frac{\partial f(x)}{\partial x_1} \quad \frac{\partial f(x)}{\partial x_2}]^T $$
$$ \frac{\partial f(x)}{\partial x_1} = 100 \times 2 (x_2-x_1^2)\times(-2x_1)-2(1-x_1)\times(-1) = 400 x_1^3 -400 x_1 x_2 - 2 + 2 x_1 $$
$$ \frac{\partial f(x)}{\partial x_2} = 100 \times 2 (x_2-x_1^2)\times(1)-2(1-x_1)\times(0) = 200 x_2 - 200 x_1^2 $$

### $$ \mathbf{H} = \begin{bmatrix} \frac{\partial^2 f}{\partial x_1^2} & \frac{\partial^2 f}{\partial x_1 \partial x_2} \\ \frac{\partial^2 f}{\partial x_2 \partial x_1} & \frac{\partial^2 f}{\partial x_2^2} \end{bmatrix} $$
$$ \frac{\partial^2 f}{\partial x_1^2} = \frac{\partial}{\partial x_1} \left( 400 x_1^3 -400 x_1 x_2 - 2 + 2 x_1 \right) = 1200x_1^2 - 400x_2 + 2 $$
$$ \frac{\partial^2 f}{\partial x_1 \partial x_2} = \frac{\partial}{\partial x_2} \left( 400 x_1^3 -400 x_1 x_2 - 2 + 2 x_1 \right) = -400x_1 $$
$$ \frac{\partial^2 f}{\partial x_2 \partial x_1} = \frac{\partial}{\partial x_1} (200 x_2 - 200 x_1^2) = -400x_1 $$
$$ \frac{\partial^2 f}{\partial x_2^2} = \frac{\partial}{\partial x_2} (200 x_2 - 200 x_1^2) = 200 $$

In [378]:
import numpy as np

def f(x):
    return 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2

def g(x):
    return np.array([-400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0]),
                             200 * (x[1] - x[0]**2)])

def h(x):
    return np.array([[-400 * (x[1] - 3 * x[0]**2) + 2, -400 * x[0]],
                     [       -400 * x[0]             ,     200]])

#### starting from $ x_0 = [-4, 10]^T $, optimize $ f(x) $ once using steepest descent and another time using Newton's method.

In [379]:
def steepest_descent(x0, lr=0.00001, tol=1e-6, max_iter=300):
    x = x0
    path = [x0]
    i = 0
    while (i < max_iter):
        i += 1
        grad = g(x)
        x_new = x - lr * grad
        if np.linalg.norm(grad) < tol:
            break
        if np.linalg.norm(grad) > 1e10:
            print("large number of gradient caused instabality in calculations and the algorithm probably would not converge.")
            break
        path.append(x_new)
        x = x_new

    if (i == max_iter):
        print("steepest descent reached maximum number of iterations.")
    else:
        print("steepest descent converged after", i, "iterations.")
    return np.array(path)

def newton_method(x0, tol=1e-6, max_iter=10):
    x = x0
    path = [x0]
    i = 0
    while (i < max_iter):
        i += 1
        grad = g(x)
        hess = h(x)
        if np.linalg.det(hess) == 0:
            print("Hessian is singular")
            return
        x_new = x - np.linalg.inv(hess).dot(grad)
        path.append(x_new)
        if np.linalg.norm(x_new - x) < tol:
            break
        x = x_new

    if (i == max_iter):
        print("Newton's method reached maximum number of iterations.")
    else:
        print("Newton's method converged after", i, "iterations.")
    return np.array(path)

x0 = np.array([-4, 10])
sd_path = steepest_descent(x0)
nm_path = newton_method(x0)

steepest descent reached maximum number of iterations.
Newton's method converged after 6 iterations.


In [380]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [381]:
fig = make_subplots(rows=1, cols=2, subplot_titles=("steepest descent", "Newton's method"))
fig.add_trace(px.scatter(x=sd_path[:, 0], y=sd_path[:, 1]).data[0], row=1, col=1)
fig.add_trace(px.scatter(x=nm_path[:, 0], y=nm_path[:, 1]).data[0], row=1, col=2)
fig.update_layout(height=600, width=1200, title_text="Optimization Paths")
fig.show()

In [382]:
sd_f = [f(x) for x in sd_path]
nm_f = [f(x) for x in nm_path]

fig_f = make_subplots(rows=1, cols=2, subplot_titles=("f(x) vs Iterations (Steepest Descent)", "f(x) vs Iterations (Newton's Method)"))
fig_f.add_trace(px.line(x=np.arange(len(sd_f)), y=sd_f).data[0], row=1, col=1)
fig_f.add_trace(px.line(x=np.arange(len(nm_f)), y=nm_f).data[0], row=1, col=2)
fig_f.update_layout(height=600, width=1200, title_text="f(x) vs Iteration Index")
fig_f.show()

In [383]:
opt = np.array([1, 1])
sd_d = [np.linalg.norm(x - opt) for x in sd_path]
nm_d = [np.linalg.norm(x - opt) for x in nm_path]

fig_o = make_subplots(rows=1, cols=2, subplot_titles=("Error vs Iterations (Steepest Descent)", "Error vs Iterations (Newton's Method)"))
fig_o.add_trace(px.line(x=np.arange(len(sd_d)), y=sd_d).data[0], row=1, col=1)
fig_o.add_trace(px.line(x=np.arange(len(nm_d)), y=nm_d).data[0], row=1, col=2)
fig_o.update_layout(height=600, width=1200, title_text="f(x) vs Iteration Index")
fig_o.show()

In [389]:
px.scatter_3d(x=sd_path[:, 0], y=sd_path[:, 1], z=sd_f)

In [384]:
px.scatter_3d(x=nm_path[:, 0], y=nm_path[:, 1], z=nm_f)

In [385]:
sd_path2 = steepest_descent(x0, max_iter=10)
nm_path2 = newton_method(sd_path2[-1])
comb_path = []
for x in sd_path2:
    comb_path.append([x[0], x[1]])

for x in nm_path2:
    comb_path.append([x[0], x[1]])

comb_path_x0, comb_path_x1 = [], []
for x in comb_path:
    comb_path_x0.append(x[0])
    comb_path_x1.append(x[1])

steepest descent reached maximum number of iterations.
Newton's method converged after 6 iterations.


In [386]:
comb_f = [f(x) for x in comb_path]
opt = np.array([1, 1])
comb_d = [np.linalg.norm(x - opt) for x in comb_path]

fig3 = make_subplots(rows=1, cols=3, subplot_titles=("x0 vs x2", "f(x) vs iterations", "error vs iterations"))
fig3.add_trace(px.scatter(x=comb_path_x0, y=comb_path_x1).data[0], row=1, col=1)
fig3.add_trace(px.line(x=np.arange(len(comb_f)), y=comb_f).data[0], row=1, col=2)
fig3.add_trace(px.line(x=np.arange(len(comb_d)), y=comb_d).data[0], row=1, col=3)
fig3.update_layout(height=600, width=1300)
fig3.show()

In [387]:
# px.scatter_3d(x=comb_path_x0, y=comb_path_x1, z=comb_f)